In [8]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Import the datetime module from the datetime library.
from datetime import datetime
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
print(coordinates)

[(43.00680128587217, -105.32388961545189), (-24.47932127775981, -149.1737141936786), (32.636007256415866, -82.7534427651125), (-4.4408462567942735, -49.234353137018985), (-54.25579877106687, 2.4071080186993186), (38.52722474764278, 60.08404876542147), (-31.061513699163775, -139.21570134052718), (-8.845330745123718, -7.815721174294964), (-86.07236650234552, -151.61970258641134), (7.6143234080996365, 77.41395924929725), (-55.528564660064454, -25.311932058438913), (72.47059553313389, -98.47293557068107), (10.35664437970236, -62.2661480838302), (48.0548472297441, -77.05841982961606), (46.43168408841217, 103.32996428472802), (-5.93696086871401, 88.56979471147696), (64.2289111029975, -159.3388641089949), (55.261734129152785, -34.86722121951712), (-68.15570640604994, 25.034827909018304), (67.92429783005664, -135.8787177616759), (81.93674794888227, -131.06895086003317), (-43.55834526870961, 6.761985782177334), (41.617283707560375, 5.015821847657605), (73.47509585551944, -151.2494963223465), (1

In [4]:
print(len(coordinates))

1500


In [5]:
print(coordinates[0])

(43.00680128587217, -105.32388961545189)


In [6]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [7]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

593

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in our list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    